In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# Stock data price 
prices_path = Path("../data/prices.csv")
df = pd.read_csv(prices_path)

### Flagging days 

In [3]:
# standardize column names for easier matching
df.columns = [c.strip() for c in df.columns]

#  detect date column
date_col_candidates = ["date", "Date", "timestamp", "Timestamp"]
date_col = next((c for c in date_col_candidates if c in df.columns), None)
if date_col is None:
    raise ValueError("Could not find a date column. Expected one of: 'date', 'Date', 'timestamp'.")

#  parse date and sort
df[date_col] = pd.to_datetime(df[date_col], utc=False).dt.tz_localize(None)
df = df.sort_values(date_col).drop_duplicates(subset=[date_col]).reset_index(drop=True)

# detect price column (prefer adjusted close if available)
price_col_candidates = ["Close/Last", "Adj Close", "Adj_Close", "adj_close", "AdjClose", "adjusted_close", "Close", "close"]
price_col = next((c for c in price_col_candidates if c in df.columns), None)
if price_col is None:
    raise ValueError("Could not find a price column. Expected one of: 'Close/Last', 'Adj Close', 'Close', etc.")

print(f"Using date column: {date_col}")
print(f"Using price column: {price_col}")
print(df[[date_col, price_col]].tail(10))

Using date column: Date
Using price column: Close/Last
           Date Close/Last
2508 2020-02-14    $324.95
2509 2020-02-18       $319
2510 2020-02-19    $323.62
2511 2020-02-20     $320.3
2512 2020-02-21    $313.05
2513 2020-02-24    $298.18
2514 2020-02-25    $288.08
2515 2020-02-26    $292.65
2516 2020-02-27    $273.52
2517 2020-02-28    $273.36


### Computing and Saving daily returns 

In [4]:
import pandas as pd
from pathlib import Path

# identify columns from earlier step (adjust if you renamed them)
date_col = "Date"
price_col = "Close/Last"

# ensure date is datetime and sorted
df[date_col] = pd.to_datetime(df[date_col], utc=False).dt.tz_localize(None)
df = df.sort_values(date_col).drop_duplicates(subset=[date_col]).reset_index(drop=True)

# clean price: remove $ and commas, cast to float
if df[price_col].dtype == "object":
    df[price_col] = (
        df[price_col]
        .astype(str)
        .str.replace("$", "", regex=False)
        .str.replace(",", "", regex=False)
        .astype(float)
    )

# compute daily returns and 20-day rolling std of returns
df["return"] = df[price_col].pct_change()
df["roll20_std"] = df["return"].rolling(window=20, min_periods=20).std()

# drop initial rows without roll20_std
df_clean = df.dropna(subset=["return", "roll20_std"]).reset_index(drop=True)

print(df_clean[[date_col, price_col, "return", "roll20_std"]].tail(10))
print("\nRows after cleaning:", len(df_clean))

           Date  Close/Last    return  roll20_std
2488 2020-02-14      324.95  0.000246    0.018433
2489 2020-02-18      319.00 -0.018311    0.018812
2490 2020-02-19      323.62  0.014483    0.018996
2491 2020-02-20      320.30 -0.010259    0.019159
2492 2020-02-21      313.05 -0.022635    0.019812
2493 2020-02-24      298.18 -0.047500    0.022403
2494 2020-02-25      288.08 -0.033872    0.022700
2495 2020-02-26      292.65  0.015864    0.021949
2496 2020-02-27      273.52 -0.065368    0.025077
2497 2020-02-28      273.36 -0.000585    0.025090

Rows after cleaning: 2498


### Marking days and saving them 20-day rolling std

In [5]:
from pathlib import Path

df_clean["is_event_day"] = (df_clean["return"].abs() > 2 * df_clean["roll20_std"])
event_days = df_clean.loc[df_clean["is_event_day"], [date_col, price_col, "return", "roll20_std"]].copy()

# Save to CSV
DATA_DIR = Path("../data")
DATA_DIR.mkdir(parents=True, exist_ok=True) # ensure data directory exists

event_path = DATA_DIR / "event_days.csv"
event_days.to_csv(event_path, index=False)

print(f"Event days saved to {event_path.resolve()}")
print(event_days.head(15))
print("\nTotal event days flagged:", len(event_days))

Event days saved to /Users/valentinreateguirangel/Documents/MSc Machine Learning/Finance_RAG_why_move/finance-rag-why-move/data/event_days.csv
          Date  Close/Last    return  roll20_std
16  2010-04-21     37.0314  0.059814    0.014981
29  2010-05-10     36.2843  0.076868    0.030566
64  2010-06-29     36.5957 -0.045212    0.019654
94  2010-08-11     35.7414 -0.035544    0.014278
109 2010-09-01     35.7614  0.029739    0.014541
132 2010-10-05     41.2771  0.036965    0.013956
140 2010-10-15     44.9628  0.041116    0.015218
160 2010-11-12     44.0043 -0.027237    0.012335
164 2010-11-18     44.0614  0.026388    0.012773
194 2011-01-03     47.0814  0.021732    0.006489
199 2011-01-10     48.9221  0.018847    0.007043
204 2011-01-18     48.6643 -0.022468    0.009145
208 2011-01-24     48.2071  0.032840    0.013271
228 2011-02-22     48.3728 -0.034089    0.013692
244 2011-03-16     47.1443 -0.044639    0.017861

Total event days flagged: 145
